# Mod 2 Assessment

### Congratulations on making it to your second assessment! Just a few reminders before you continue:
- This should only take ~70 Minutes, so be sure to manage your time effectively.
- Read the instructions carefully for _specified variable names_.

If there is any confusion on a question, please ask for clarification from an instructor. </br>Though we can't give you the answer, we can help clear up any misunderstandings and get you back on track.

In [85]:
# You'll need these imports to start.
import pymongo
from bson.json_util import loads
from collections import Counter

# put all of your additional imports here:

import pandas as pd
from bs4 import BeautifulSoup as bs
import sqlite3
import requests


## Section 0: Coding Readibility (5 Minutes)
Fix the errors in the code below, and rewrite using best practices (hint: don't forget to consider improving variable names).  The final output should BOTH print all variables AND return the list of all the tuples:
```python
def blabla():
for i in range ( 10 ):for j in range( 10 ):for k in range(10): print( i,j,k ) return ( i,j,k )
```

In [68]:
def blabla():
    
    answer = []
    
    for i in range(10):
        for j in range(10):
            for k in range(10):
                print(i, j, k)
                answer.append((i, j, k))
    return answer

## Section 1: SQL (10 Minutes)
There is a sqlite3 database in `assets/books.db`. The SQL to create this is also in `assets/books.sql`; if you want to run it manually you can also import this using https://sqliteonline.com/, or run the SQL file directly.  Both have the same schema and data.

The schema has three tables. You can explore the schema in the file posted above.  Please answer the following questions.

Connect to the database using sqlite3. <br/>
Assign `conn` to the sqlite3 connection <br/>
Assign `cur`  to the connection's cursor object

In [69]:
!ls assets

books.db  books.sql  grades.jsonl


In [70]:
conn = sqlite3.connect('assets/books.db')
cur = conn.cursor()

#### Querying the DB: 

1. How many pages are in the book "Nine Stories"?

In [71]:
# Assign `answer_1` to your final answer
statement = """

SELECT pages
FROM book
WHERE title="Nine Stories"

"""

cur.execute(statement)
info = cur.fetchall()

answer_1 = info[0][0]
print(answer_1)

600


2. How many authors are from the USA?

In [72]:
# Assign `answer_2` to your final answer
statement = """

SELECT COUNT(*)
FROM author
WHERE country="USA"

"""

cur.execute(statement)
info = cur.fetchall()
print(*info, sep = "\n") 

answer_2 = info[0][0]
print(answer_2)

(6,)
6


3. How many authors does the book "Professional ASP.NET 4.5 in C# and VB" have?

In [73]:
# Assign `answer_3` to your final answer
statement = """

SELECT COUNT(ba.author_id)
FROM book_author ba
JOIN book b ON ba.book_id=b.book_id
WHERE b.book_id=7

"""

cur.execute(statement)
info = cur.fetchall()
print(*info, sep = "\n")
answer_3 = info[0][0]

(5,)


4. How many pages total have been written by non-American authors?

In [74]:
# Assign `answer_4` to your final answer

statement = """

SELECT SUM(b.pages)
FROM book b
JOIN (
    SELECT ba.author_id, ba.book_id, a.country
    FROM book_author ba
    JOIN author a ON ba.author_id=a.author_id
    WHERE a.country!="USA"
    ) joined
ON b.book_id=joined.book_id

"""

cur.execute(statement)
info = cur.fetchall()
print(*info, sep = "\n")

answer_4 = info[0][0]
print(answer_4)

(30003,)
30003


## Section 2: Object Oriented Programming (15 Minutes)

### Creating a Class
1. Force every new instance of `WeWorkMember` to expect a value which is assigned to the `name` **attribute**.
1. Give every new instance of `WeWorkMember` a `caffeinated` **attribute** that is set to `False`. 
1. Give `WeWorkMember` an **instance method** called `caffeinate()` that prints out "Getting coffee!" and sets the `caffeinated` attribute to `True`.

In [75]:
class WeWorkMember:
    
    def __init__(self, name):
        self.name = name
        self.caffeinated = False
        
    def caffeinate(self):
        print("Getting coffee!")
        self.caffeinated = True

### Inheriting from a Class

1. Have `Staff` and `Student` inherit all methods from `WeWorkMember`
1. Give `Staff` a **static method** called `cheer()` that prints out "Goooooooo Flatiron Students!"
1. Give `Students` a **class method** called `learn()` that takes in an integer and returns that number +1

In [76]:
class Staff(WeWorkMember):
    
    @staticmethod
    def cheer():
        print("Goooooooo Flatiron Students!")

class Student(WeWorkMember):
    
    @classmethod
    def learn(cls, num):
        if type(num)==int:
            return num+1
        else:
            raise TypeError

In [79]:
Andy = Staff('Andy')
Andrew = Student('Student')

## Section 3: APIs & Web Scraping
### APIs (10 Minutes)
Using the API about RuPaul's Drag Race, tell me how many judges of each **`type`** there were in the **first 50 records** the API returns.

API: http://www.nokeynoshade.party/api/judges </br>
Docs: https://drag-race-api.readme.io/docs/get-all-judges

 - Assign `rupaul_resp` to the response of the API request.
 - Ensure the request only returns the first 50 records; the judges will have **`id`** ranging from 1 to 50
 - Do the aggregation in pure Python, Pandas, or SQL -- whatever's easiest for you
 - Assign `judge_count` to a dictionary with the number of judges for each type

In [112]:
parameters = {'limit': 50}

rupaul_resp = requests.get("http://www.nokeynoshade.party/api/judges", params=parameters)

new_json = rupaul_resp.json()

def aggregate_types(json):
    answer = list()
    
    for judge in json:
        answer.append(judge['type'])
        
    return answer

new_series = pd.Series(aggregate_types(new_json))
judge_type_counts = new_series.value_counts()

judge_type_counts['guest']

47

In [113]:
judge_count = {}

for index_name in judge_type_counts.index:
    judge_count[index_name] = judge_type_counts[index_name]
    
print(judge_count)

{'guest': 47, 'regular': 2, 'interim': 1}


### Web Scraping (15 Minutes)
Scrape the website [http://books.toscrape.com/](http://books.toscrape.com/) and save it to the `book_resp` variable. Then use BeautifulSoup to convert the response to data we can work with.

- Set the response to a variable named `book_resp`
- Set the BeautifulSoup() object to a variable named `book_soup`

In [160]:
book_resp = requests.get("http://books.toscrape.com/")

soup = bs(book_resp.content, 'html.parser')
books = soup.find_all("li", class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")

list_soup = soup.find_all("li", class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")

new_col1 = []

for soup_object in list_soup:
    new_col1.append(soup_object.find("a"))
    
new_series = pd.Series(new_col1)

## Section 4: NoSQL (10 Minutes)

#### Load data from `assets/grades.json` into Mongo 

(this code is written for you)

In [128]:
# you shouldn't need to edit this cell!
db_name = "mod2db"

with open('assets/grades.jsonl') as f:
    # loads() comes from the bson library
    file_data = [loads(line) for line in f.readlines()]

client = pymongo.MongoClient("mongodb://localhost:27017/")
client.drop_database(db_name)
db = client[db_name]
coll = db["testcoll"]

coll.insert_many(file_data)

#### Answer all of the following questions by querying Mongo and manipulating the results in Python

1. How many records are there total?

In [130]:
nosql_answer1 = coll.count_documents({})

2. How many students have taken the class with `class_id` = **29**?

In [132]:
# Set `nosql_answer2` to your final answer
nosql_answer2 = coll.count_documents({"class_id": 29})

9


3. </br>For student **12** in class **23**, what grade did they get on their exam?

In [140]:
# Set `nosql_answer3` to your final answer
student_12_class_23 = coll.find_one({"student_id": 12, "class_id": 23})

for score_dict in student_12_class_23['scores']:
    if score_dict['type']=='exam':
        answer = score_dict['score']
        break
        
nosql_answer3 = answer

print(nosql_answer3)

26.9857216299485


In [16]:
# for when you're done with this portion, this deletes the data we added.
client.drop_database("mod2db")

## Assessment submission (2 Minutes)
Please save your completed file as `mod2_assessment.ipynb` (UNCHANGED) and upload it using [this form](https://docs.google.com/forms/d/e/1FAIpQLSeY0nH0-DF4sQTQ8vBEsjOwLiQaB-JWopmhh7kFef3TOscrWA/viewform?usp=sf_link)